In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from astropy.table import Table, vstack
from astroquery.vizier import Vizier
from astropy import units as u

import swasputils

In [3]:
from IPython.display import clear_output

In [4]:
CATALOGUES = [
    'J/BaltA/9/646/catalog',
    'J/ApJS/247/44/table2',
]

In [5]:
source_table = Table.read(os.path.join(swasputils.CACHE_LOCATION, 'source_coords.fits'))

In [6]:
SPLIT_SIZE = 1000
total_iterations = int(len(source_table) / SPLIT_SIZE) + 1

In [7]:
# Vizier messes up the _q index when you query multiple catalogues at once
for catalogue_count, catalogue in enumerate(CATALOGUES, start=1):
    results = []
    for i in range(total_iterations):
        clear_output(wait=True)
        print(f' [{catalogue_count}/{len(CATALOGUES)}] Querying {catalogue}: {i + 1}/{total_iterations}')
        sources = source_table[i * SPLIT_SIZE : (i+1) * SPLIT_SIZE]
        if len(sources) > 0:
            try:
                results.append(Vizier.query_region(sources, radius=2*u.arcmin, catalog=catalogue)[0])
            except IndexError:
                continue
            results[-1].add_column(
                [ sources['SuperWASP Coords'][i - 1] for i in results[-1]['_q'] ],
                name='SuperWASP Coords',
            )
    if results:
        results = vstack(results)
        catalogue = catalogue.replace('/', '_')
        results.write(os.path.join(swasputils.CACHE_LOCATION, f'catalogue_match_{catalogue}.ecsv'), overwrite=True)

 [2/2] Querying J/ApJS/247/44/table2: 771/771
